In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from google.cloud import bigquery
import datetime
from google.cloud.bigquery.client import Client
import pandas as pd
import os

In [2]:
osp='G:/hedden/asu-msba-free-trial-conversion-credentials.json'
Project='infusionsoft-looker-poc'
def get_data_from_bigQ(Project,osp):##osp is the place hold the CREDENTIALS
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = osp
    sqlbigquery_data_app = """SELECT * FROM `infusionsoft-looker-poc.asu_msba_free_trial_conversion.CONFIDENTIAL_free_trail_apps_table` """
    free_Trial_app_data= pd.read_gbq(sqlbigquery_data_app,project_id=Project,dialect='standard')
    sqlbigquery_data_usage = """SELECT u.* FROM `infusionsoft-looker-poc.asu_msba_free_trial_conversion.CONFIDENTIAL_usage_data` u
        join `infusionsoft-looker-poc.asu_msba_free_trial_conversion.CONFIDENTIAL_free_trail_apps_table`  a
        on u.appname = a.app_name 
        where u.appname not in ('tl471','wd410','mw416','kw563','ov450') and u.date BETWEEN DATE (a.trial_date, "America/Los_Angeles") and 
        IF(date_diff(Date(new_customer_date, "America/Los_Angeles"), Date(trial_date, "America/Los_Angeles"),day)<=13,date(a.new_customer_date,"America/Los_Angeles") ,DATE_ADD( date(a.trial_date,"America/Los_Angeles"), interval 13 DAY))"""
    usage_dataset_free= pd.read_gbq(sqlbigquery_data_usage,
                  project_id='infusionsoft-looker-poc',
                  dialect='standard')
    return free_Trial_app_data,usage_dataset_free

In [3]:
Free,Usage=get_data_from_bigQ(Project,osp)

In [102]:
def dropDup(df,cat): ## delete the duplicate caused by join
    dfN = df.groupby(['app_name'])[[cat]].apply(lambda x: x.sort_values([cat], ascending=False).iloc[0, :])
    return dfN
def floatc(x):
    if pd.isna(x):
        return np.nan
    else:
        return float(x)
def first_day_engin(All_data):
    action=[x for x in All_data.columns.tolist() if x not in ['appname','trial_date','date','NUM_CONTACTS','free_email']]
    All_data['Heavy_contact_update']=All_data['CONTACTS_UPDATED'].apply(lambda x:1 if x>10 else 0)
    All_data['Heavy_LOGIN_COUNT']=All_data['LOGIN_COUNT'].apply(lambda x:1 if x>10 else 0)
    All_data['Heavy_CONTACT_GROUP_COUNT']=All_data['CONTACT_GROUP_COUNT'].apply(lambda x:1 if x>40 else 0)
    All_data['Heavy_CONTACTS_ADDED_AUTO']=All_data['CONTACTS_ADDED_AUTO'].apply(lambda x:1 if x>40 else 0)
    All_data['Heavy_CONTACTS_ADDED_OTHER']=All_data['CONTACTS_ADDED_OTHER'].apply(lambda x:1 if x>900 else 0)
    All_data['Heavy_WEB_ANALYTICS_NUM_PAGES']=All_data['WEB_ANALYTICS_NUM_PAGES'].apply(lambda x:1 if x>40 else 0)
    All_data['Heavy_NUMEMAILSSENT_AUTO']=All_data['NUMEMAILSSENT_AUTO'].apply(lambda x:1 if x>100 else 0)
    All_data['Heavy_NUMEMAILSSENT_MANUAL']=All_data['NUMEMAILSSENT_MANUAL'].apply(lambda x:1 if x>10 else 0)
    All_data['total_action']=All_data[action].sum(axis=1)
    All_data['Heavy_total_action']=All_data['total_action'].apply(lambda x:1 if x>1.300000e+01 else 0)
    email=['NUMEMAILSSENT_AUTO', 'NUMEMAILSSENT_MANUAL',
        'NUMEMAILSSENT_AUTO_SYSTEM',
       'NUMEMAILS_RECEIVED',
       'NUMEMAILSOPENED_AUTO', 
       'NUMEMAILSOPENED_MANUAL', 
       'NUMEMAILSOPENED_BROADCAST', 
       'NUMEMAILSOPENED_AUTO_SYSTEM', 
       'NUMEMAILSOPENED_NULL', 
       'NUMEMAILSCLICKED_AUTO', 
       'NUMEMAILSCLICKED_MANUAL',  
       'NUMEMAILSCLICKED_NULL',]
    contact=['NUMCONTACTSSENT_AUTO','NUMCONTACTSSENT_MANUAL','NUMCONTACTSSENT_AUTO_SYSTEM','NUMCONTACTS_RECEIVED'
        ,'NUMCONTACTSOPENED_AUTO','NUMCONTACTSOPENED_MANUAL','NUMCONTACTSOPENED_BROADCAST','NUMCONTACTSOPENED_AUTO_SYSTEM',
        'NUMCONTACTSCLICKED_AUTO','NUMCONTACTSCLICKED_MANUAL'
        , 'CONTACTS_UPDATED','CONTACT_GROUP_COUNT',
       'CONTACTS_ADDED_AUTO', 'CONTACTS_ADDED_OTHER','TOTAL_CONTACTS_ADDED_AUTO', 'TOTAL_CONTACTS_ADDED_OTHER']
    Web=['WEBFORM_COUNT', 'WEBFORM_NUM_FORMS', 'WEBFORM_NEW_CONTACTS',
       'WEBFORM_REFERRING_DOMAINS','WEB_ANALYTICS_NUM_PAGES', 'WEB_ANALYTICS_NUM_PAGEVIEWS',
       'WEB_ANALYTICS_UNIQUE_VISITORS', 'WEB_ANALYTICS_UNIQUE_CONTACTS',
       'WEB_ANALYTICS_UNIQUE_CUSTOMERS', 'WEB_ANALYTICS_TOTAL_VIEWS']
    All_data['total_email']=All_data[email].sum(axis=1)
    All_data['total_contact']=All_data[contact].sum(axis=1)
    All_data['total_Web']=All_data[Web].sum(axis=1)
    All_data['email_user']=All_data['total_email'].apply(lambda x: 1 if x>0 else 0)
    All_data['heavy_total_contact']=All_data['total_contact'].apply(lambda x: 1 if x>1.000000e+01 else 0)
    return All_data[['appname','Heavy_contact_update','Heavy_LOGIN_COUNT','Heavy_CONTACT_GROUP_COUNT','Heavy_CONTACTS_ADDED_AUTO','Heavy_CONTACTS_ADDED_OTHER','Heavy_WEB_ANALYTICS_NUM_PAGES','Heavy_NUMEMAILSSENT_AUTO','Heavy_NUMEMAILSSENT_MANUAL','Heavy_total_action','total_email','total_contact','total_Web','email_user','heavy_total_contact']]

def data_prepare(Free,Usage):
    Free_C=Free.copy();Usage_C=Usage.copy()
    Free_C.drop_duplicates(keep="first", inplace=True)
    Usage_C.drop_duplicates(keep="first", inplace=True)
    DFL=[]
    for cat in ['first_contact_on', 'contact_lead_source', 'contact_phase', 'lead_lead_source', 'lead_converted_date']:
        Data=Free_C[['app_name',cat]]
        dfN = dropDup(Data,cat)
        DFL.append(dfN)
    DFree_N = pd.concat(DFL, axis=1)
    DFree_N['app_name'] = DFree_N.index
    DFree_N.reset_index(drop=True,inplace=True)
    AP = Free_C[['app_name', 'is_free_trial_initiated', 'trial_date', 'new_customer_date']]
    AP.drop_duplicates(inplace=True)
    Free_final = AP.merge(DFree_N, left_on=['app_name'], right_on=['app_name'])
    Free_final['Target'] = list(map((lambda x: 0 if pd.isna(x) else 1),Free_final['new_customer_date']))
    from sklearn.impute import SimpleImputer
    si_usage=SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)
    Usage_nm = pd.DataFrame(si_usage.fit_transform(Usage_C),index=Usage_C.index,columns=Usage_C.columns)
    Usage_nm.loc[:,'invoice_amount'] = Usage_nm['invoice_amount'].apply(floatc)
    
    mergedata_target = pd.merge(Usage_nm,Free_final[['app_name','Target']],how='left',left_on='appname',right_on='app_name')
    mergedata_target.drop('app_name',axis=1,inplace=True)
    sample_usage_sorted=mergedata_target.sort_values(['appname','date'],ascending=True)
    sample_usage_first3days=mergedata_target.sort_values(['appname','date'],ascending=True).groupby('appname').head(3)
    sample_usage_firstday=mergedata_target.sort_values(['appname','date'],ascending=True).groupby('appname').head(1)
    fisrt_usage_data= first_day_engin(sample_usage_firstday)
    
    features_tobeengineered=['CONTACTS_UPDATED','NUMEMAILSSENT_AUTO','NUMEMAILSSENT_MANUAL','NUMEMAILS_RECEIVED','NUMEMAILSSENT_AUTO_SYSTEM',
   'TOTAL_CONTACTS_ADDED_OTHER','TOTAL_CONTACTS_ADDED_AUTO','NUMCONTACTSSENT_AUTO_SYSTEM','LOGIN_COUNT', 
                        'USER_LOGINS', 'merchant_infu_created', 'funnel_created', 'broadcasts_created'
                        ,'actions_created']
    sample_usage_subset=sample_usage_first3days[['appname','date']+features_tobeengineered]
    sample_usage_first3days_sum=sample_usage_first3days.groupby(['appname','free_email']).sum().reset_index()
    
    sample_usage_1stday=sample_usage_subset.sort_values(['appname','date'],ascending=True).groupby('appname').nth(0).reset_index()
    sample_usage_2ndday=sample_usage_subset.sort_values(['appname','date'],ascending=True).groupby('appname').nth(1).reset_index()
    sample_usage_3rdday=sample_usage_subset.sort_values(['appname','date'],ascending=True).groupby('appname').nth(2).reset_index()
    sample_usage_1stday=sample_usage_1stday.add_suffix('_1stday')
    sample_usage_1stday.rename(columns={'appname_1stday':'appname'},inplace=True )
    sample_usage_2ndday=sample_usage_2ndday.add_suffix('_2ndday')
    sample_usage_2ndday.rename(columns={'appname_2ndday':'appname'},inplace=True )
    sample_usage_3rdday=sample_usage_3rdday.add_suffix('_3rdday')
    sample_usage_3rdday.rename(columns={'appname_3rdday':'appname'},inplace=True )
    df_final=sample_usage_1stday.merge(sample_usage_2ndday,on='appname',how='left').merge(sample_usage_3rdday,on='appname',how='left')
    df_final=df_final.fillna(0)
    for each in features_tobeengineered:
        df_final[each+"_diff1stand2nd"]=df_final[each+'_2ndday']-df_final[each+'_1stday']
        df_final[each+"_diff1stand2nd_trend"]=list(map((lambda x1,x2: "no change" if (x1==x2 and x1==0.0) else " change"),df_final[each+'_2ndday'],df_final[each+'_1stday']))
        df_final[each+"_diff2ndand3rd"]=df_final[each+'_3rdday']-df_final[each+'_2ndday']
        df_final[each+"_diff2ndand3rd_trend"]=list(map((lambda x1,x2: "no change" if (x1==x2 and x1==0.0) else " change"),df_final[each+'_3rdday'],df_final[each+'_2ndday']))
    sample_usage_first3days_final=sample_usage_first3days_sum.merge(df_final,on='appname',how='left')
    sample_usage_first3days_final=sample_usage_first3days_final.drop(['Target'],axis=1)
    
    mergedata = pd.merge(sample_usage_first3days_final,Free_final,how='left',left_on='appname',right_on='app_name')
    mergedata.fillna("missing",inplace=True)
    mergedata_final=pd.merge(mergedata,fisrt_usage_data,how='left',left_on='appname',right_on='appname')
    return mergedata_final

In [103]:
final_data=data_prepare(Free,Usage)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [104]:
def split_train_test_bytime(df,train_size=0.8):
    n = df.shape[0]
    df = df.sort_values('trial_date')
    train_dataframe = df.iloc[:int(n * train_size)]
    test_dataframe = df.iloc[int(n * train_size):]
    X_test=train_dataframe['Target']
    X_train=train_dataframe.drop('Target',axis=1)
    y_test=test_dataframe['Target']
    y_train=test_dataframe.drop('Target',axis=1)
    return X_train.copy(), X_test.copy(), y_train.copy(), y_test.copy()
def split_train_test_byrandom(df,random_state_=1,test_size_=.2):
    y=df['Target']
    X=df.drop('Target',axis=1)
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=random_state_,test_size=test_size_)
    return  X_train.copy(),X_test.copy(), y_train.copy(),y_test.copy()

In [105]:
X_train_R, X_test_R, y_train_R, y_test_R=split_train_test_byrandom(final_data)
X_train_D, X_test_D, y_train_D, y_test_D=split_train_test_bytime(final_data)

In [106]:
def get_kvalue_train(col,df):
    from sklearn.cluster import KMeans
    from sklearn.feature_extraction.text import TfidfVectorizer
    from scipy.spatial.distance import cdist
    vectorizer = TfidfVectorizer(min_df = 1,lowercase = False, ngram_range = (1,1), use_idf = True, stop_words='english')
    #XTtrain = pd.DataFrame(tv.fit_transform(Xtrain).toarray(), columns=tv.get_feature_names())
    X = pd.DataFrame(vectorizer.fit_transform(df[col]).toarray(),columns=vectorizer.get_feature_names())
    #X = X.toarray()
    distortions=[]
    K = range(1,10)
    for k in K:
        kmeanModel = KMeans(n_clusters=k).fit(X)
        kmeanModel.fit(X)
        distortions.append(sum(np.min(cdist(X, kmeanModel.cluster_centers_, 'euclidean'), axis=1)) / X.shape[0])
    return X,vectorizer
def get_kvalue_test(col,df,vec):
    X =  pd.DataFrame(vec.transform(df[col]).toarray(),columns=vec.get_feature_names())
    #X = X.toarray()
    return X
def get_clusters_train(df,X,col,k):
    from sklearn.cluster import KMeans
    km = KMeans(n_clusters=k, random_state=0)
    km.fit_transform(X)
    cluster_labels = km.labels_
    cluster_labels = pd.DataFrame(cluster_labels, columns=['ClusterLabel_'+col],index=df.index)
    cluster_labels
    pd.concat([df[col], cluster_labels],axis=1)
    df2=pd.concat([df, cluster_labels],axis=1)
    return df2,km
def get_clusters_test(df,X,col,km):
    km.transform(X)
    cluster_labels = km.predict(X)
    cluster_labels = pd.DataFrame(cluster_labels, columns=['ClusterLabel_'+col],index=df.index) 
    pd.concat([df[col], cluster_labels],axis=1)
    df2=pd.concat([df, cluster_labels],axis=1)
    return df2 
def convertUrltocluster_train(colname,df):
    colname_new=colname+'_modified'
    df[colname_new]=df[colname].str.strip().str.split('[\W_]+')
    df[colname_new]=df[colname_new].astype(str)
    X,vec=get_kvalue_train(colname_new,df)
    return X,colname_new,vec
def convertUrltocluster_test(colname,df,vec):
    colname_new=colname+'_modified'
    df[colname_new]=df[colname].str.strip().str.split('[\W_]+')
    df[colname_new]=df[colname_new].astype(str)
    X=get_kvalue_test(colname_new,df,vec)
    return X,colname_new

In [111]:
def cal_number_of_days(dataset): 
    ndays_columns_set=set()
    final_datefeatures=['lead_converted_date_imp','date_1stday', 'date_2ndday','date_3rdday','first_contact_on']
    for each in final_datefeatures:
        ndayseach='ndays_'+each
        ndays_columns_set.add(ndayseach)
        dataset[ndayseach]=(dataset[each].dt.date-dataset['trial_date'].dt.date).dt.days
        dataset[ndayseach]=dataset[ndayseach].apply(lambda x: 3 if x>3 else x)
        return dataset
def phase(x):## get the phase number
    if pd.isna(x) or x=='None' or x=='missing': return 0
    else:
        m=x.split(' ')[1]
        return int(m)
def get_prepare_final_data(X_train,X_test,k=4):
    X_train_X,colname_new,vec=convertUrltocluster_train('lead_lead_source',X_train)
    X_test_X,colname_new=convertUrltocluster_test('lead_lead_source',X_test,vec)
    X_train2,km=get_clusters_train(X_train,X_train_X,colname_new,k)
    X_test2=get_clusters_test(X_test,X_test_X,colname_new,km)
    X_train2_X,colname_new,vec=convertUrltocluster_train('contact_lead_source',X_train2)
    X_test2_X,colname_new=convertUrltocluster_test('contact_lead_source',X_test2,vec)
    X_train3,km=get_clusters_train(X_train2,X_train2_X,colname_new,4)
    X_test3=get_clusters_test(X_test2,X_test2_X,colname_new,km)
    X_train3['lead_converted_date_imp']=X_train3['lead_converted_date'].copy()
    X_test3['lead_converted_date_imp']=X_test3['lead_converted_date'].copy()
    X_train3.drop(['lead_converted_date'],inplace=True,axis=1)
    X_test3.drop(['lead_converted_date'],inplace=True,axis=1)
    X_train3['date_2ndday'].replace(0, datetime.datetime.now(),inplace=True)
    X_train3['date_3rdday'].replace(0, datetime.datetime.now(),inplace=True)
    X_test3['date_2ndday'].replace(0, datetime.datetime.now(),inplace=True)
    X_test3['date_3rdday'].replace(0, datetime.datetime.now(),inplace=True)
    X_train3['first_contact_on'].replace(0, datetime.datetime.now(),inplace=True)
    X_test3['first_contact_on'].replace(0, datetime.datetime.now(),inplace=True)
    for c in ['lead_converted_date_imp','date_1stday', 'date_2ndday','date_3rdday','first_contact_on']:
        X_train3[c] = pd.to_datetime(X_train3[c])
        X_test3[c] = pd.to_datetime(X_test3[c])
    X_train3=cal_number_of_days(X_train3)
    X_test3=cal_number_of_days(X_test3)
    X_train3['contact_phase_Scale'] = X_train3['contact_phase'].apply(phase)
    X_test3['contact_phase_Scale'] = X_test3['contact_phase'].apply(phase)
    drop_feature=['appname','first_contact_on','trial_date','date_1stday','date_2ndday',
                  'date_3rdday','lead_converted_date_imp','app_name','lead_lead_source',
                  'lead_lead_source_modified', 'contact_lead_source_modified','contact_lead_source','contact_phase']
    X_train5=X_train3.drop(drop_feature,axis=1)
    X_test5=X_test3.drop(drop_feature,axis=1)
    X_train5=X_train5.drop(['new_customer_date'],axis=1)
    X_test5=X_test5.drop(['new_customer_date'],axis=1)
    dtypes = pd.DataFrame(X_train5.dtypes.rename('type')).reset_index().astype('str')
    numericorig = dtypes[(dtypes.type.isin(['int64','float64']))]['index'].values
    categoricalorig = dtypes[~(dtypes['index'].isin(numericorig))]['index'].values
    numeric_final1=numericorig.tolist()
    categoricalorig_final1=categoricalorig.tolist()
    from sklearn.preprocessing import StandardScaler
    ss = StandardScaler(with_mean=True, with_std=True)
    X_trainnumss = pd.DataFrame(ss.fit_transform(X_train5[numeric_final1]), columns=['ss_'+x for x in numeric_final1],index=X_train5.index)
    X_trainnumss = pd.concat([X_train5, X_trainnumss], axis=1)
    X_trainnumss = X_trainnumss.drop(numeric_final1, axis=1)
    X_testnumss = pd.DataFrame(ss.transform(X_test5[numeric_final1]), columns=['ss_'+x for x in numeric_final1],index=X_test5.index)
    X_testnumss = pd.concat([X_test5, X_testnumss], axis=1)
    X_testnumss = X_testnumss.drop(numeric_final1, axis=1)
    import category_encoders as ce
    ohe = ce.OneHotEncoder(handle_unknown='ignore', use_cat_names=True)
    X_trainss = pd.DataFrame(ohe.fit_transform(X_trainnumss[categoricalorig_final1].astype(str)), index=X_trainnumss.index)
    X_trainss = pd.concat([X_trainnumss, X_trainss], axis=1)
    X_trainss = X_trainss.drop(categoricalorig_final1, axis=1)
    X_testss=pd.DataFrame(ohe.transform(X_testnumss[categoricalorig_final1].astype(str)),index=X_testnumss.index)
    X_testss = pd.concat([X_testnumss, X_testss], axis=1)
    X_testss = X_testss.drop(categoricalorig_final1, axis=1)
    X_trainss=X_trainss.fillna(0)
    X_testss= X_testss.fillna(0)
    return X_trainss,X_testss

In [112]:
X_trainss,X_testss=get_prepare_final_data(X_train_R,X_test_R)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:58: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.


In [ ]:
X_trainss=X_trainss.fillna(0)
X_testss= X_testss.fillna(0)

In [113]:
main_list = np.setdiff1d(X_trainss.columns,X_testss.columns)
main_list

array([], dtype=object)

In [ ]:
from sklearn import svm, metrics, tree, decomposition
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier 
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier, OrthogonalMatchingPursuit, RandomizedLogisticRegression 
from sklearn.neighbors.nearest_centroid import NearestCentroid 
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
    def define_model( model, n_cores = 0):
        clfs = {'RandomForestClassifier': RandomForestClassifier(random_state=10),
                'ExtraTreesClassifier': ExtraTreesClassifier(),
                'GradientBoostingClassifier': GradientBoostingClassifier(random_state=10),
                'GaussianNB': GaussianNB(),
                'KNeighborsClassifier': KNeighborsClassifier()}
        spaces={'GradientBoostingClassifier':[Integer(5,30,name="max_depth"),
                                         Integer(10,90,name="n_estimators"),
                                         Real(10**-5, 10**0, "log-uniform", name='learning_rate'),
                                         Integer(1, n_features, name='max_features'),
                                         Integer(2, 100, name='min_samples_split'),
                                         Integer(1, 100, name='min_samples_leaf')],
          'RandomForestClassifier':[Integer(5,90,name="max_depth"),
                                    Integer(50,250,name="n_estimators"),
                                    Integer(2, 100, name='min_samples_split'),
                                    Integer(1, 100, name='min_samples_leaf')],
          'GaussianNB':[Real(10**-10, 10**-5,name='var_smoothing')],
          'KNeighborsClassifier': [ Integer(3, 7, name='n_neighbors')],
           'ExtraTreesClassifier':[Integer(50,250,name="n_estimators"),
                                    Integer(2, 100, name='min_samples_split'),
                                    Integer(1, 100, name='min_samples_leaf')]}
        clf = clfs[model]
        space=spaces[model]
        return clf,space

In [ ]:
P_score=-np.mean(cross_val_score(model,X_trainss,y_train,cv=5,n_jobs=-1,scoring='precision' ))

In [83]:
from skopt import gp_minimize

In [84]:
from skopt.space import Real,Integer,Categorical
from skopt.utils import use_named_args
from sklearn.model_selection import cross_val_score
from sklearn.metrics import fbeta_score, make_scorer
f2_5_scorer = make_scorer(fbeta_score, beta=2.5)
n_features=X_trainss.shape[1]

In [115]:
score_dic={}
for modeln in ['RandomForestClassifier','ExtraTreesClassifier','GradientBoostingClassifier']:
    model,space=define_model(modeln)
    @use_named_args(space)
    def objective_F(**params):
        model.set_params(**params)
        return -np.mean(cross_val_score(model,X_trainss,y_train_R,cv=5,n_jobs=-1,scoring=f2_5_scorer ))
    result_F25= gp_minimize(objective_F,space,n_calls=50,random_state=0)
    @use_named_args(space)
    def objective_P(**params):
        model.set_params(**params)
        return -np.mean(cross_val_score(model,X_trainss,y_train_R,cv=5,n_jobs=-1,scoring='precision' ))
    result_P=gp_minimize(objective_P,space,n_calls=50,random_state=0)
    score_dic[modeln]={'F':result_F25,'P':result_P}
    print('%s'%modeln ,' the score of F is %.4f' %result_F25.fun)
    print('%s'%modeln ,' the score of F is %.4f' %result_P.fun)

C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\ProgramData\

RandomForestClassifier  the score of F is -0.5843
RandomForestClassifier  the score of F is -0.8000


C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\ProgramData\

ExtraTreesClassifier  the score of F is -0.5745
ExtraTreesClassifier  the score of F is -0.8594
GradientBoostingClassifier  the score of F is -0.5777
GradientBoostingClassifier  the score of F is -0.7512


In [151]:
for modeln in ['GaussianNB','KNeighborsClassifier']:
    model,space=define_model(modeln)
    @use_named_args(space)
    def objective_F(**params):
        model.set_params(**params)
        return -np.mean(cross_val_score(model,X_trainss,y_train_R,cv=5,n_jobs=-1,scoring=f2_5_scorer ))
    result_F25= gp_minimize(objective_F,space,n_calls=10,random_state=0)
    @use_named_args(space)
    def objective_P(**params):
        model.set_params(**params)
        return -np.mean(cross_val_score(model,X_trainss,y_train_R,cv=5,n_jobs=-1,scoring='precision' ))
    result_P=gp_minimize(objective_P,space,n_calls=10,random_state=0)
    score_dic[modeln]={'F':result_F25,'P':result_P}
    print('%s'%modeln ,' the score of F is %.4f' %result_F25.fun)
    print('%s'%modeln ,' the score of F is %.4f' %result_P.fun)

GaussianNB  the score of F is -0.3671
GaussianNB  the score of F is -0.0762
KNeighborsClassifier  the score of F is -0.5665
KNeighborsClassifier  the score of F is -0.6929


In [95]:
import pandas_profiling as pp

In [143]:
def get_model( model,result):
    NO=[0,0,0,0,0,0]
    result=result.x+NO
    clfs = {'RandomForestClassifier': RandomForestClassifier(
        max_depth=result[0],n_estimators=result[1],min_samples_split=result[2],
        min_samples_leaf=result[3],random_state=10),
            'ExtraTreesClassifier': ExtraTreesClassifier(n_estimators=result[0],min_samples_split=result[1],min_samples_leaf=result[2]),
            'GradientBoostingClassifier': GradientBoostingClassifier(max_depth=result[0],n_estimators=result[1],learning_rate=result[2],max_features=result[3],min_samples_split=result[4],min_samples_leaf=result[5],random_state=10),
            'GaussianNB': GaussianNB(var_smoothing=result[0]),
            'KNeighborsClassifier': KNeighborsClassifier(n_neighbors=result[0])}
    clf=clfs[model]
    return clf
       

In [147]:
from sklearn.metrics import classification_report

In [148]:
Train_dic={};Test_dic={}
i=1
for alg in ['RandomForestClassifier','GradientBoostingClassifier','ExtraTreesClassifier']:
    alg_dic=score_dic[alg]
    result_F=alg_dic['F']
    result_F=alg_dic['P']
    clf_F=get_model(alg,result_F)
    clf_P=get_model(alg,result_P)
    clf_F.fit(X_trainss,y_train_R)
    clf_P.fit(X_trainss,y_train_R)
    train_Pred_F=clf_F.predict(X_trainss)
    train_Pred_P=clf_P.predict(X_trainss)
    train_predprob_F= clf_F.predict_proba(X_trainss)[:,1]
    train_predprob_P= clf_P.predict_proba(X_trainss)[:,1]
    test_Pred_F=clf_F.predict(X_testss)
    test_Pred_P=clf_P.predict(X_testss)
    test_predprob_F= clf_F.predict_proba(X_testss)[:,1]
    test_predprob_P= clf_P.predict_proba(X_testss)[:,1]
    Train_dic[alg+'F']=train_Pred_F
    Train_dic[alg+'P']=train_Pred_P
    Test_dic[alg+'F']=test_Pred_F
    Test_dic[alg+'P']=test_Pred_P
    print(alg,'Parameter',result_F.x,"Fscore train:",classification_report(y_train_R,train_Pred_F))
    print(alg,"Fscore test:",classification_report(y_test_R,test_Pred_F))
    print(alg,'Parameter',result_P.x,"Pscore train:",classification_report(y_train_R,train_Pred_P))
    print(alg,"Pscore test:",classification_report(y_test_R,test_Pred_P))
    i=i+1

RandomForestClassifier Parameter [5, 229, 73, 100] Fscore train:               precision    recall  f1-score   support

           0       0.94      1.00      0.97     21447
           1       0.79      0.07      0.13      1443

   micro avg       0.94      0.94      0.94     22890
   macro avg       0.86      0.53      0.55     22890
weighted avg       0.93      0.94      0.92     22890

RandomForestClassifier Fscore test:               precision    recall  f1-score   support

           0       0.94      1.00      0.97      5368
           1       0.66      0.06      0.11       355

   micro avg       0.94      0.94      0.94      5723
   macro avg       0.80      0.53      0.54      5723
weighted avg       0.92      0.94      0.92      5723

RandomForestClassifier Parameter [27, 57, 0.0199796750842793, 52, 36, 65] Pscore train:               precision    recall  f1-score   support

           0       0.95      0.99      0.97     21447
           1       0.71      0.23      0.35     

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


ExtraTreesClassifier Parameter [27, 57, 0.0199796750842793, 52, 36, 65] Pscore train:               precision    recall  f1-score   support

           0       0.94      1.00      0.97     21447
           1       0.00      0.00      0.00      1443

   micro avg       0.94      0.94      0.94     22890
   macro avg       0.47      0.50      0.48     22890
weighted avg       0.88      0.94      0.91     22890



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


ExtraTreesClassifier Pscore test:               precision    recall  f1-score   support

           0       0.94      1.00      0.97      5368
           1       0.00      0.00      0.00       355

   micro avg       0.94      0.94      0.94      5723
   macro avg       0.47      0.50      0.48      5723
weighted avg       0.88      0.94      0.91      5723



In [152]:
for alg in ['GaussianNB','KNeighborsClassifier']:
    alg_dic=score_dic[alg]
    result_F=alg_dic['F']
    result_F=alg_dic['P']
    clf_F=get_model(alg,result_F)
    clf_P=get_model(alg,result_P)
    clf_F.fit(X_trainss,y_train_R)
    clf_P.fit(X_trainss,y_train_R)
    train_Pred_F=clf_F.predict(X_trainss)
    train_Pred_P=clf_P.predict(X_trainss)
    train_predprob_F= clf_F.predict_proba(X_trainss)[:,1]
    train_predprob_P= clf_P.predict_proba(X_trainss)[:,1]
    test_Pred_F=clf_F.predict(X_testss)
    test_Pred_P=clf_P.predict(X_testss)
    test_predprob_F= clf_F.predict_proba(X_testss)[:,1]
    test_predprob_P= clf_P.predict_proba(X_testss)[:,1]
    Train_dic[alg+'F']=train_Pred_F
    Train_dic[alg+'P']=train_Pred_P
    Test_dic[alg+'F']=test_Pred_F
    Test_dic[alg+'P']=test_Pred_P
    print(alg,'Parameter',result_F.x,"Fscore train:",classification_report(y_train_R,train_Pred_F))
    print(alg,"Fscore test:",classification_report(y_test_R,test_Pred_F))
    print(alg,'Parameter',result_P.x,"Pscore train:",classification_report(y_train_R,train_Pred_P))
    print(alg,"Pscore test:",classification_report(y_test_R,test_Pred_P))

GaussianNB Parameter [8.579470381665808e-06] Fscore train:               precision    recall  f1-score   support

           0       0.99      0.23      0.38     21447
           1       0.08      0.95      0.14      1443

   micro avg       0.28      0.28      0.28     22890
   macro avg       0.53      0.59      0.26     22890
weighted avg       0.93      0.28      0.36     22890

GaussianNB Fscore test:               precision    recall  f1-score   support

           0       0.99      0.24      0.38      5368
           1       0.08      0.97      0.14       355

   micro avg       0.28      0.28      0.28      5723
   macro avg       0.53      0.60      0.26      5723
weighted avg       0.93      0.28      0.37      5723

GaussianNB Parameter [4] Pscore train:               precision    recall  f1-score   support

           0       0.94      0.98      0.96     21447
           1       0.19      0.06      0.09      1443

   micro avg       0.93      0.93      0.93     22890
   mac

In [153]:
Stack_Train=pd.DataFrame.from_dict(Train_dic)
Stack_Test=pd.DataFrame.from_dict(Test_dic)

In [154]:
def stack_learn(x, y,test_x):
    # set sample weight
    clf = GradientBoostingClassifier(
        loss='deviance',random_state=10).fit(x,y)
    prediction_list = clf.predict(test_x)
    y_predprob= clf.predict_proba(test_x)[:,1]
    return prediction_list,y_predprob,clf
pred_test_y_stack,y_predprob_stack,GB=stack_learn(Stack_Train,y_train_R,Stack_Test)
print(classification_report(y_test_R,pred_test_y_stack))

              precision    recall  f1-score   support

           0       0.96      0.99      0.97      5368
           1       0.68      0.44      0.54       355

   micro avg       0.95      0.95      0.95      5723
   macro avg       0.82      0.71      0.76      5723
weighted avg       0.95      0.95      0.95      5723



In [161]:
n_features=Stack_Train.shape[1]
space = [Integer(5,30,name="max_depth"),
         Integer(10,90,name="n_estimators"),
         Real(10**-5, 10**0, "log-uniform", name='learning_rate'),
         Integer(1, n_features, name='max_features'),
         Integer(2, 100, name='min_samples_split'),
         Integer(1, 100, name='min_samples_leaf')]
model = GradientBoostingClassifier(random_state=10)
@use_named_args(space)
def objective_P(**params):
    model.set_params(**params)
    return -np.mean(cross_val_score(model,Stack_Train,y_train_R,cv=5,n_jobs=-1,scoring='precision' ))
result_P = gp_minimize(objective_P,space,n_calls=50,random_state=0)
clf = GradientBoostingClassifier(
    loss='deviance',learning_rate=result_P.x[2],n_estimators=result_P.x[1], max_depth=result_P.x[0],min_samples_leaf=result_P.x[5], max_features=result_P.x[3],min_samples_split=result_P.x[4], subsample=0.8,random_state=10)
clf=clf.fit(Stack_Train,y_train_R)
train_Pred_P=clf.predict(Stack_Train)
test_Pred_P=clf.predict(Stack_Test)


NameError: name 'test_Pred_R' is not defined

In [162]:
print(classification_report(y_train_R,train_Pred_P))
print(classification_report(y_test_R,test_Pred_P))

              precision    recall  f1-score   support

           0       0.96      0.99      0.98     21447
           1       0.78      0.44      0.56      1443

   micro avg       0.96      0.96      0.96     22890
   macro avg       0.87      0.72      0.77     22890
weighted avg       0.95      0.96      0.95     22890

              precision    recall  f1-score   support

           0       0.96      0.99      0.97      5368
           1       0.68      0.39      0.50       355

   micro avg       0.95      0.95      0.95      5723
   macro avg       0.82      0.69      0.74      5723
weighted avg       0.94      0.95      0.94      5723



In [ ]:
f1_5_scorer = make_scorer(fbeta_score, beta=1.5)
n_features=X_trainss.shape[1]
score_dic2={}
for modeln in ['GaussianNB','KNeighborsClassifier','RandomForestClassifier','GradientBoostingClassifier','ExtraTreesClassifier']:
    model,space=define_model(modeln)
    @use_named_args(space)
    def objective_F(**params):
        model.set_params(**params)
        return -np.mean(cross_val_score(model,X_trainss,y_train_R,cv=5,n_jobs=-1,scoring=f1_5_scorer ))
    result_F15= gp_minimize(objective_F,space,n_calls=50,random_state=10)
    @use_named_args(space)
    def objective_P(**params):
        model.set_params(**params)
        return -np.mean(cross_val_score(model,X_trainss,y_train_R,cv=5,n_jobs=-1,scoring='precision' ))
    result_P=gp_minimize(objective_P,space,n_calls=50,random_state=10)
    score_dic2[modeln]={'F':result_F25,'P':result_P}
    print('%s'%modeln ,'parameter:',result_F15.x,' the score of F is %.4f' %result_F25.fun)
    print('%s'%modeln ,'parameter:',result_P.x,' the score of F is %.4f' %result_P.fun)

C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\ProgramData\

C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\ProgramData\

GaussianNB parameter: [9.998525975362338e-06]  the score of F is -0.5953
GaussianNB parameter: [9.998525975362338e-06]  the score of F is -0.0765


C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\ProgramData\

C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\ProgramData\

KNeighborsClassifier parameter: [5]  the score of F is -0.5953
KNeighborsClassifier parameter: [4]  the score of F is -0.6929


C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\ProgramData\

RandomForestClassifier parameter: [60, 250, 100, 1]  the score of F is -0.5953
RandomForestClassifier parameter: [5, 123, 2, 100]  the score of F is -0.8245


In [ ]:
def result_and_stack(model,dic):
    result_F=alg_dic['F']
    result_F=alg_dic['P']
    clf_F=get_model(alg,result_F)
    clf_P=get_model(alg,result_P)
    clf_F.fit(X_trainss,y_train_R)
    clf_P.fit(X_trainss,y_train_R)
    train_Pred_F=clf_F.predict(X_trainss)
    train_Pred_P=clf_P.predict(X_trainss)
    train_predprob_F= clf_F.predict_proba(X_trainss)[:,1]
    train_predprob_P= clf_P.predict_proba(X_trainss)[:,1]
    test_Pred_F=clf_F.predict(X_testss)
    test_Pred_P=clf_P.predict(X_testss)
    test_predprob_F= clf_F.predict_proba(X_testss)[:,1]
    test_predprob_P= clf_P.predict_proba(X_testss)[:,1]
    print(alg,'Parameter',result_F.x,"Fscore train:",classification_report(y_train_R,train_Pred_F))
    print(alg,"Fscore test:",classification_report(y_test_R,test_Pred_F))
    print(alg,'Parameter',result_P.x,"Pscore train:",classification_report(y_train_R,train_Pred_P))
    print(alg,"Pscore test:",classification_report(y_test_R,test_Pred_P))
    return train_Pred_F,train_Pred_P,test_Pred_F,test_Pred_P

In [ ]:

for alg in ['RandomForestClassifier','GradientBoostingClassifier','ExtraTreesClassifier','GaussianNB','KNeighborsClassifier']:
    alg_dic=score_dic2[alg]
    Train_dic[alg+'F'],Train_dic[alg+'P'],Test_dic[alg+'F'],Test_dic[alg+'P']=result_and_stack(alg,alg_dic)
    #result_F=alg_dic['F']
    #result_F=alg_dic['P']
    #clf_F=get_model(alg,result_F)
    #clf_P=get_model(alg,result_P)
    #clf_F.fit(X_trainss,y_train_R)
    #clf_P.fit(X_trainss,y_train_R)
    #train_Pred_F=clf_F.predict(X_trainss)
    #train_Pred_P=clf_P.predict(X_trainss)
    #train_predprob_F= clf_F.predict_proba(X_trainss)[:,1]
    #train_predprob_P= clf_P.predict_proba(X_trainss)[:,1]
    #test_Pred_F=clf_F.predict(X_testss)
    #test_Pred_P=clf_P.predict(X_testss)
    #test_predprob_F= clf_F.predict_proba(X_testss)[:,1]
    #test_predprob_P= clf_P.predict_proba(X_testss)[:,1]
    #Train_dic[alg+'F']=train_Pred_F
    #Train_dic[alg+'P']=train_Pred_P
    #Test_dic[alg+'F']=test_Pred_F
    #Test_dic[alg+'P']=test_Pred_P
    #print(alg,'Parameter',result_F.x,"Fscore train:",classification_report(y_train_R,train_Pred_F))
    #print(alg,"Fscore test:",classification_report(y_test_R,test_Pred_F))
    #print(alg,'Parameter',result_P.x,"Pscore train:",classification_report(y_train_R,train_Pred_P))
    #print(alg,"Pscore test:",classification_report(y_test_R,test_Pred_P))

In [169]:
score_dic2={}
for alg in ['RandomForestClassifier','GradientBoostingClassifier','ExtraTreesClassifier']:
    alg_dic=score_dic2[alg]
    result_F=alg_dic['F']
    result_F=alg_dic['P']
    clf_F=get_model(alg,result_F)
    clf_P=get_model(alg,result_P)
    clf_F.fit(X_trainss,y_train_R)
    clf_P.fit(X_trainss,y_train_R)
    train_Pred_F=clf_F.predict(X_trainss)
    train_Pred_P=clf_P.predict(X_trainss)
    train_predprob_F= clf_F.predict_proba(X_trainss)[:,1]
    train_predprob_P= clf_P.predict_proba(X_trainss)[:,1]
    test_Pred_F=clf_F.predict(X_testss)
    test_Pred_P=clf_P.predict(X_testss)
    test_predprob_F= clf_F.predict_proba(X_testss)[:,1]
    test_predprob_P= clf_P.predict_proba(X_testss)[:,1]
    Train_dic[alg+'F']=train_Pred_F
    Train_dic[alg+'P']=train_Pred_P
    Test_dic[alg+'F']=test_Pred_F
    Test_dic[alg+'P']=test_Pred_P
    print(alg,'Parameter',result_F.x,"Fscore train:",classification_report(y_train_R,train_Pred_F))
    print(alg,"Fscore test:",classification_report(y_test_R,test_Pred_F))
    print(alg,'Parameter',result_P.x,"Pscore train:",classification_report(y_train_R,train_Pred_P))
    print(alg,"Pscore test:",classification_report(y_test_R,test_Pred_P))

KeyError: 'RandomForestClassifier'